In [2]:
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'langchain'

In [ ]:
import os

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import OnlinePDFLoader, PagedPDFSplitter, TextLoader, UnstructuredPDFLoader

def get_context(file_path: str, prompt: str) -> str:

    # Load the document
    loader = PagedPDFSplitter(file_path)
    doc = loader.load()

    paragraphs = []
    for page in doc:
        paragraphs.extend(page.paragraphs)

    # Split the document into sentences
    splitter = RecursiveCharacterTextSplitter()
    sentences = splitter.split(doc)

    # Embed the sentences
    embeddings = OpenAIEmbeddings()
    embedded_sentences = embeddings.embed(sentences)

    # Create a vector store
    store = Chroma()

    # Create a language model
    lm = OpenAI()

    # Create a QA chain
    chain = VectorDBQA(store, lm)

    # Add the embedded sentences to the vector store
    for sentence, embedding in zip(sentences, embedded_sentences):
        store.add(sentence, embedding)

    # Ask the QA chain a question
    return chain.ask(prompt)

In [ ]:
file_path = "1706.03762.pdf"

In [ ]:
# Load the document
loader = PagedPDFSplitter(file_path)
doc = loader.load()

In [ ]:
len(paragraphs)

In [ ]:
paragraphs = [p for p in doc.split("1706.03762.pdf") if len(p) > 0]

In [ ]:
loader = PagedPDFSplitter("1706.03762.pdf")
pages = loader.load_and_split()

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
Chroma()

In [ ]:
store = Chroma()

In [ ]:
store.add_documents(pages)

In [ ]:
embeddings = OpenAIEmbeddings()


In [ ]:
from chromadb.utils import embedding_functions

In [ ]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.environ["OPENAI_API_KEY"],
                model_name=""
            )

In [ ]:
openai_ef("Hello world")

In [ ]:
Chroma(embedding_function=openai_ef)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("How will the community be engaged?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

In [ ]:
data[0].paragraphs

In [ ]:
loader = PagedPDFSplitter("1706.03762.pdf")
data = loader.load()

In [ ]:
type(data[0])

In [ ]:
data[0]

In [ ]:
len(data)

In [ ]:
get_context(file_path="1706.03762.pdf", prompt='What is the main idea of the paper?')

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [ ]:
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

In [ ]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=vectordb)


In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)